<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/PAPER_GDRNet_ELECTRICITY_Step4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from keras import backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping
import keras.backend as K
from keras.callbacks import Callback
import plotly
import plotly.express as px # for data visualization
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import numpy as np
import scipy.stats as stats
from numpy import array
from numpy import hstack
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV, MultiTaskLassoCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import TimeSeriesSplit
from math import sqrt
from sklearn.linear_model import LinearRegression
from keras.models import Model
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.stattools import acf

#-----------------------------------------------------GET DATA - X, Predicted Wndow from MLP, and Derived Labels, for the entire dataset that was gennerated through VAE-----------------------------------

#-----------for whole dataset-----------------------------------------------------------------------
y_pred = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/predicted_COSW_V3.csv')
y_true = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/Calculated_label_COSW_V3.csv')
x_orig = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/whole_data_V3.csv')


#-----------for test dataset-----------------------------------------------------------------------
y_pred = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/predicted_window_V3.csv')
y_true = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/derived_window_label_V3.csv')
x_orig = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/test_data_V3.csv')

n_features = 6
window_size = int(x_orig.shape[1]/n_features)
x = x_orig.reshape((x_orig.shape[0],window_size,n_features))


#-----------------------------------------------CASES - pot window values and pick a few fixed window sizes -------------------------------------------



plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.title("Distribution before Transformation", fontsize=15)
sns.histplot(y_true, kde=True, color="red")
plt.subplot(1,2,2)


fx_window1 = 6
fx_window2 = 7
fx_window3 = 8


pred_step = 1

############################METHOD 1---------------- GO BY AIC ---------------------------------------------------------------------------------

aic1_count = list()
aic2_count = list()
aic3_count = list()
aic4_count = list()
aic5_count = list()
aic6_count = list()





for i in range(x.shape[0]-1) :

  x1 = x[i]
  x_train=x1[:-pred_step]
  x_test=x1[-pred_step:]
  regressor= VAR(x_train)
  results1 = regressor.fit(fx_window1)
  results2 = regressor.fit(fx_window2)
  results3 = regressor.fit(fx_window3)
  #results4 = regressor.fit(fx_window4)
  #results5 = regressor.fit(fx_window5)
  results6 = regressor.fit(int(y_pred[i]))

#Calculate AIC for ach case--------------
  try:
    aic1 = results1.aic
  except:
    aic1 = 9999
  aic1_count.append(aic1)

  try:
    aic2 = results2.aic
  except:
    aic2 = 9999
  aic2_count.append(aic2)

  try:
    aic3 = results3.aic
  except:
    aic3 = 9999
  aic3_count.append(aic3)

  #try:
   # aic4 = results4.aic
  #except:
   # aic4 = 9999
  #aic4_count.append(aic4)

  #try:
  #  aic5 = results5.aic
  #except:
  #  aic5 = 9999
  #aic5_count.append(aic5)


  try:
    aic6 = results6.aic
  except:
    aic6 = 9999
  aic6_count.append(aic6)


print("aic1_count : ", aic1_count)
print("aic2_count : ", aic2_count)
print("aic3_count : ", aic3_count)
#print("aic4_count : ", aic4_count)
#print("aic5_count : ", aic5_count)
print("aic6_count : ", aic6_count)

df = pd.DataFrame({'AIC1' : aic1_count, 'AIC2' : aic2_count, 'AIC3' : aic3_count,   'AIC6 - Predicted' : aic6_count})
df[0:1000].plot(figsize=(15,6))
plt.legend()
plt.show()

df.to_csv('/content/drive/MyDrive/PHD/2024/PREDICTION_COMPARISON/AIC_comparison.csv', index = False)

################---------------------------METHOD 2- -GO BY RMSE----------------------------------------------------####################

def forecast_accuracy(forecast, actual):
    mse = np.mean((forecast - actual)**2)  # MSE
    return mse

#------------------------------------V1----------------------------------------

mse_11 = list()
mse_12 = list()
mse_13 = list()
mse_1X = list()

sample_size = 1000
for i in range(sample_size) :
  x1 = x[i]
  df = pd.DataFrame(x1, columns=['V1','V2','V3','V4','V5','V6'])
  df_train, df_test = df[0:-pred_step], df[-pred_step:]
  model= VAR(df_train)

  model_fitted1 = model.fit(fx_window1)
  lag_order1 = model_fitted1.k_ar
  forecast_input1 = df.values[-lag_order1:]
  fc1 = model_fitted1.forecast(y=forecast_input1, steps=pred_step)
  df_forecast1 = pd.DataFrame(fc1, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecast1['V1'].values, df_test['V1'])
  mse_11.append(mse)

  model_fitted2 = model.fit(fx_window2)
  lag_order2 = model_fitted2.k_ar
  forecast_input2 = df.values[-lag_order2:]
  fc2 = model_fitted2.forecast(y=forecast_input2, steps=pred_step)
  df_forecast2 = pd.DataFrame(fc2, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecast2['V1'].values, df_test['V1'])
  mse_12.append(mse)

  model_fitted3 = model.fit(fx_window3)
  lag_order3 = model_fitted3.k_ar
  forecast_input3 = df.values[-lag_order3:]
  fc3 = model_fitted3.forecast(y=forecast_input3, steps=pred_step)
  df_forecast3 = pd.DataFrame(fc3, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecast3['V1'].values, df_test['V1'])
  mse_13.append(mse)

  model_fittedX = model.fit(int(y_pred[i]))
  lag_orderX = model_fittedX.k_ar
  forecast_inputX = df.values[-lag_orderX:]
  fcX = model_fittedX.forecast(y=forecast_inputX, steps=pred_step)
  df_forecastX = pd.DataFrame(fcX, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecastX['V1'].values, df_test['V1'])
  mse_1X.append(mse)

overall_mse_11 = sum(mse_11)/len(mse_11)
overall_mse_12 = sum(mse_12)/len(mse_12)
overall_mse_13 = sum(mse_13)/len(mse_13)
overall_mse_1X = sum(mse_1X)/len(mse_1X)

norm_mse_11 = overall_mse_11/(max(mse_11)-min(mse_11))
norm_mse_12 = overall_mse_12/(max(mse_12)-min(mse_12))
norm_mse_13 = overall_mse_13/(max(mse_13)-min(mse_13))
norm_mse_1X = overall_mse_1X/(max(mse_1X)-min(mse_1X))

print(norm_mse_11)
print(norm_mse_12)
print(norm_mse_13)
print(norm_mse_1X)


#------------------------------------V2----------------------------------------

mse_21 = list()
mse_22 = list()
mse_23 = list()
mse_2X = list()

sample_size = 1000
for i in range(sample_size) :
  x1 = x[i]
  df = pd.DataFrame(x1, columns=['V1','V2','V3','V4','V5','V6'])
  df_train, df_test = df[0:-pred_step], df[-pred_step:]
  model= VAR(df_train)

  model_fitted1 = model.fit(fx_window1)
  lag_order1 = model_fitted1.k_ar
  forecast_input1 = df.values[-lag_order1:]
  fc1 = model_fitted1.forecast(y=forecast_input1, steps=pred_step)
  df_forecast1 = pd.DataFrame(fc1, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecast1['V2'].values, df_test['V2'])
  mse_21.append(mse)

  model_fitted2 = model.fit(fx_window2)
  lag_order2 = model_fitted2.k_ar
  forecast_input2 = df.values[-lag_order2:]
  fc2 = model_fitted2.forecast(y=forecast_input2, steps=pred_step)
  df_forecast2 = pd.DataFrame(fc2, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecast2['V2'].values, df_test['V2'])
  mse_22.append(mse)

  model_fitted3 = model.fit(fx_window3)
  lag_order3 = model_fitted3.k_ar
  forecast_input3 = df.values[-lag_order3:]
  fc3 = model_fitted3.forecast(y=forecast_input3, steps=pred_step)
  df_forecast3 = pd.DataFrame(fc3, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecast3['V2'].values, df_test['V2'])
  mse_23.append(mse)

  model_fittedX = model.fit(int(y_pred[i]))
  lag_orderX = model_fittedX.k_ar
  forecast_inputX = df.values[-lag_orderX:]
  fcX = model_fittedX.forecast(y=forecast_inputX, steps=pred_step)
  df_forecastX = pd.DataFrame(fcX, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecastX['V2'].values, df_test['V2'])
  mse_2X.append(mse)

overall_mse_21 = sum(mse_21)/len(mse_21)
overall_mse_22 = sum(mse_22)/len(mse_22)
overall_mse_23 = sum(mse_23)/len(mse_23)
overall_mse_2X = sum(mse_2X)/len(mse_2X)

norm_mse_21 = overall_mse_21/(max(mse_21)-min(mse_21))
norm_mse_22 = overall_mse_22/(max(mse_22)-min(mse_22))
norm_mse_23 = overall_mse_23/(max(mse_23)-min(mse_23))
norm_mse_2X = overall_mse_2X/(max(mse_2X)-min(mse_2X))

print(norm_mse_21)
print(norm_mse_22)
print(norm_mse_23)
print(norm_mse_2X)

#------------------------------------V3----------------------------------------

mse_31 = list()
mse_32 = list()
mse_33 = list()
mse_3X = list()

sample_size = 1000
for i in range(sample_size) :
  x1 = x[i]
  df = pd.DataFrame(x1, columns=['V1','V2','V3','V4','V5','V6'])
  df_train, df_test = df[0:-pred_step], df[-pred_step:]
  model= VAR(df_train)

  model_fitted1 = model.fit(fx_window1)
  lag_order1 = model_fitted1.k_ar
  forecast_input1 = df.values[-lag_order1:]
  fc1 = model_fitted1.forecast(y=forecast_input1, steps=pred_step)
  df_forecast1 = pd.DataFrame(fc1, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecast1['V3'].values, df_test['V3'])
  mse_31.append(mse)

  model_fitted2 = model.fit(fx_window2)
  lag_order2 = model_fitted2.k_ar
  forecast_input2 = df.values[-lag_order2:]
  fc2 = model_fitted2.forecast(y=forecast_input2, steps=pred_step)
  df_forecast2 = pd.DataFrame(fc2, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecast2['V3'].values, df_test['V3'])
  mse_32.append(mse)

  model_fitted3 = model.fit(fx_window3)
  lag_order3 = model_fitted3.k_ar
  forecast_input3 = df.values[-lag_order3:]
  fc3 = model_fitted3.forecast(y=forecast_input3, steps=pred_step)
  df_forecast3 = pd.DataFrame(fc3, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecast3['V3'].values, df_test['V3'])
  mse_33.append(mse)

  model_fittedX = model.fit(int(y_pred[i]))
  lag_orderX = model_fittedX.k_ar
  forecast_inputX = df.values[-lag_orderX:]
  fcX = model_fittedX.forecast(y=forecast_inputX, steps=pred_step)
  df_forecastX = pd.DataFrame(fcX, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecastX['V3'].values, df_test['V3'])
  mse_3X.append(mse)

overall_mse_31 = sum(mse_31)/len(mse_31)
overall_mse_32 = sum(mse_32)/len(mse_32)
overall_mse_33 = sum(mse_33)/len(mse_33)
overall_mse_3X = sum(mse_3X)/len(mse_3X)

norm_mse_31 = overall_mse_31/(max(mse_31)-min(mse_31))
norm_mse_32 = overall_mse_32/(max(mse_32)-min(mse_32))
norm_mse_33 = overall_mse_33/(max(mse_33)-min(mse_33))
norm_mse_3X = overall_mse_3X/(max(mse_3X)-min(mse_3X))

print(norm_mse_31)
print(norm_mse_32)
print(norm_mse_33)
print(norm_mse_3X)

#------------------------------------V4----------------------------------------

mse_41 = list()
mse_42 = list()
mse_43 = list()
mse_4X = list()

sample_size = 1000
for i in range(sample_size) :
  x1 = x[i]
  df = pd.DataFrame(x1, columns=['V1','V2','V3','V4','V5','V6'])
  df_train, df_test = df[0:-pred_step], df[-pred_step:]
  model= VAR(df_train)

  model_fitted1 = model.fit(fx_window1)
  lag_order1 = model_fitted1.k_ar
  forecast_input1 = df.values[-lag_order1:]
  fc1 = model_fitted1.forecast(y=forecast_input1, steps=pred_step)
  df_forecast1 = pd.DataFrame(fc1, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecast1['V4'].values, df_test['V4'])
  mse_41.append(mse)

  model_fitted2 = model.fit(fx_window2)
  lag_order2 = model_fitted2.k_ar
  forecast_input2 = df.values[-lag_order2:]
  fc2 = model_fitted2.forecast(y=forecast_input2, steps=pred_step)
  df_forecast2 = pd.DataFrame(fc2, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecast2['V4'].values, df_test['V4'])
  mse_42.append(mse)

  model_fitted3 = model.fit(fx_window3)
  lag_order3 = model_fitted3.k_ar
  forecast_input3 = df.values[-lag_order3:]
  fc3 = model_fitted3.forecast(y=forecast_input3, steps=pred_step)
  df_forecast3 = pd.DataFrame(fc3, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecast3['V4'].values, df_test['V4'])
  mse_43.append(mse)

  model_fittedX = model.fit(int(y_pred[i]))
  lag_orderX = model_fittedX.k_ar
  forecast_inputX = df.values[-lag_orderX:]
  fcX = model_fittedX.forecast(y=forecast_inputX, steps=pred_step)
  df_forecastX = pd.DataFrame(fcX, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecastX['V4'].values, df_test['V4'])
  mse_4X.append(mse)

overall_mse_41 = sum(mse_41)/len(mse_41)
overall_mse_42 = sum(mse_42)/len(mse_42)
overall_mse_43 = sum(mse_43)/len(mse_43)
overall_mse_4X = sum(mse_4X)/len(mse_4X)

norm_mse_41 = overall_mse_41/(max(mse_41)-min(mse_41))
norm_mse_42 = overall_mse_42/(max(mse_42)-min(mse_42))
norm_mse_43 = overall_mse_43/(max(mse_43)-min(mse_43))
norm_mse_4X = overall_mse_4X/(max(mse_4X)-min(mse_4X))

print(norm_mse_41)
print(norm_mse_42)
print(norm_mse_43)
print(norm_mse_4X)

#------------------------------------V5----------------------------------------

mse_51 = list()
mse_52 = list()
mse_53 = list()
mse_5X = list()

sample_size = 1000
for i in range(sample_size) :
  x1 = x[i]
  df = pd.DataFrame(x1, columns=['V1','V2','V3','V4','V5','V6'])
  df_train, df_test = df[0:-pred_step], df[-pred_step:]
  model= VAR(df_train)

  model_fitted1 = model.fit(fx_window1)
  lag_order1 = model_fitted1.k_ar
  forecast_input1 = df.values[-lag_order1:]
  fc1 = model_fitted1.forecast(y=forecast_input1, steps=pred_step)
  df_forecast1 = pd.DataFrame(fc1, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecast1['V5'].values, df_test['V5'])
  mse_51.append(mse)

  model_fitted2 = model.fit(fx_window2)
  lag_order2 = model_fitted2.k_ar
  forecast_input2 = df.values[-lag_order2:]
  fc2 = model_fitted2.forecast(y=forecast_input2, steps=pred_step)
  df_forecast2 = pd.DataFrame(fc2, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecast2['V5'].values, df_test['V5'])
  mse_52.append(mse)

  model_fitted3 = model.fit(fx_window3)
  lag_order3 = model_fitted3.k_ar
  forecast_input3 = df.values[-lag_order3:]
  fc3 = model_fitted3.forecast(y=forecast_input3, steps=pred_step)
  df_forecast3 = pd.DataFrame(fc3, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecast3['V5'].values, df_test['V5'])
  mse_53.append(mse)

  model_fittedX = model.fit(int(y_pred[i]))
  lag_orderX = model_fittedX.k_ar
  forecast_inputX = df.values[-lag_orderX:]
  fcX = model_fittedX.forecast(y=forecast_inputX, steps=pred_step)
  df_forecastX = pd.DataFrame(fcX, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecastX['V5'].values, df_test['V5'])
  mse_5X.append(mse)

overall_mse_51 = sum(mse_51)/len(mse_51)
overall_mse_52 = sum(mse_52)/len(mse_52)
overall_mse_53 = sum(mse_53)/len(mse_53)
overall_mse_5X = sum(mse_5X)/len(mse_5X)

norm_mse_51 = overall_mse_51/(max(mse_51)-min(mse_51))
norm_mse_52 = overall_mse_52/(max(mse_52)-min(mse_52))
norm_mse_53 = overall_mse_53/(max(mse_53)-min(mse_53))
norm_mse_5X = overall_mse_5X/(max(mse_5X)-min(mse_5X))

print(norm_mse_51)
print(norm_mse_52)
print(norm_mse_53)
print(norm_mse_5X)

#------------------------------------V6----------------------------------------

mse_61 = list()
mse_62 = list()
mse_63 = list()
mse_6X = list()

sample_size = 1000
for i in range(sample_size) :
  x1 = x[i]
  df = pd.DataFrame(x1, columns=['V1','V2','V3','V4','V5','V6'])
  df_train, df_test = df[0:-pred_step], df[-pred_step:]
  model= VAR(df_train)

  model_fitted1 = model.fit(fx_window1)
  lag_order1 = model_fitted1.k_ar
  forecast_input1 = df.values[-lag_order1:]
  fc1 = model_fitted1.forecast(y=forecast_input1, steps=pred_step)
  df_forecast1 = pd.DataFrame(fc1, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecast1['V6'].values, df_test['V6'])
  mse_61.append(mse)

  model_fitted2 = model.fit(fx_window2)
  lag_order2 = model_fitted2.k_ar
  forecast_input2 = df.values[-lag_order2:]
  fc2 = model_fitted2.forecast(y=forecast_input2, steps=pred_step)
  df_forecast2 = pd.DataFrame(fc2, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecast2['V6'].values, df_test['V6'])
  mse_62.append(mse)

  model_fitted3 = model.fit(fx_window3)
  lag_order3 = model_fitted3.k_ar
  forecast_input3 = df.values[-lag_order3:]
  fc3 = model_fitted3.forecast(y=forecast_input3, steps=pred_step)
  df_forecast3 = pd.DataFrame(fc3, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecast3['V6'].values, df_test['V6'])
  mse_63.append(mse)

  model_fittedX = model.fit(int(y_pred[i]))
  lag_orderX = model_fittedX.k_ar
  forecast_inputX = df.values[-lag_orderX:]
  fcX = model_fittedX.forecast(y=forecast_inputX, steps=pred_step)
  df_forecastX = pd.DataFrame(fcX, index=df.index[-pred_step:], columns=df.columns)
  mse = forecast_accuracy(df_forecastX['V6'].values, df_test['V6'])
  mse_6X.append(mse)

overall_mse_61 = sum(mse_61)/len(mse_61)
overall_mse_62 = sum(mse_62)/len(mse_62)
overall_mse_63 = sum(mse_63)/len(mse_63)
overall_mse_6X = sum(mse_6X)/len(mse_6X)

norm_mse_61 = overall_mse_61/(max(mse_61)-min(mse_61))
norm_mse_62 = overall_mse_62/(max(mse_62)-min(mse_62))
norm_mse_63 = overall_mse_63/(max(mse_63)-min(mse_63))
norm_mse_6X = overall_mse_6X/(max(mse_6X)-min(mse_6X))

print(norm_mse_61)
print(norm_mse_62)
print(norm_mse_63)
print(norm_mse_6X)


0.02778741580795989
0.028120939160841064
0.02921579919594337
0.027610061593747477


In [ ]:
from google.colab import drive
drive.mount('/content/drive')